In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("Data/SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])

X_features = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)
X_features.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,49,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,62,3.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,28,7.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,135,4.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X_features, data['label'], test_size=0.2)

In [7]:
def train_RF(n_est, depth):
    rf = RandomForestClassifier(n_estimators=n_est, max_depth=depth, n_jobs=-1)
    rf_model = rf.fit(X_train, y_train) 
    y_pred = rf_model.predict(X_test)
    precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average='binary')
    print(f'Est: {n_est} | Depth: {depth} ---- Precision: {round(precision, 3)} | Recall: {round(recall, 3)} | Accuracy: {round((y_test==y_pred).sum()/len(y_pred)*100, 3)}')

In [8]:
for n_est in [10, 50, 100]:
    for depth in [10, 20, 30, None]:
        train_RF(n_est, depth)
        

Est: 10 | Depth: 10 ---- Precision: 1.0 | Recall: 0.288 | Accuracy: 91.113
Est: 10 | Depth: 20 ---- Precision: 0.987 | Recall: 0.547 | Accuracy: 94.255
Est: 10 | Depth: 30 ---- Precision: 0.989 | Recall: 0.662 | Accuracy: 95.691
Est: 10 | Depth: None ---- Precision: 0.991 | Recall: 0.791 | Accuracy: 97.307
Est: 50 | Depth: 10 ---- Precision: 1.0 | Recall: 0.223 | Accuracy: 90.305
Est: 50 | Depth: 20 ---- Precision: 1.0 | Recall: 0.568 | Accuracy: 94.614
Est: 50 | Depth: 30 ---- Precision: 1.0 | Recall: 0.683 | Accuracy: 96.05
Est: 50 | Depth: None ---- Precision: 1.0 | Recall: 0.842 | Accuracy: 98.025
Est: 100 | Depth: 10 ---- Precision: 1.0 | Recall: 0.245 | Accuracy: 90.575
Est: 100 | Depth: 20 ---- Precision: 1.0 | Recall: 0.576 | Accuracy: 94.704
Est: 100 | Depth: 30 ---- Precision: 1.0 | Recall: 0.741 | Accuracy: 96.768
Est: 100 | Depth: None ---- Precision: 1.0 | Recall: 0.849 | Accuracy: 98.115
